## Imports Overview

The following libraries are used in this project:

- **praw**: For interacting with the Reddit API.
- **oracledb**: For connecting to and working with Oracle databases.
- **os**, **cohere**, **requests**: For system operations, natural language processing, and handling HTTP requests, respectively.


In [12]:
import praw
import oracledb
import os, cohere, requests
import random

## Reddit API Configuration

In [13]:
user_agent = 'Scraper 1.0 by /u/hamza'
reddit = praw.Reddit(
    client_id="3zHMmumWQmV75yrYY6iKpg",
    client_secret="gkawOwbO_KBJvfkuMahKfHgN2TNGVQ",
    user_agent=user_agent
)

## Oracle Database Connection

In [21]:
conn = oracledb.connect(
    user="HAMZA",
    password="Oracle21c",
    dsn="localhost:1521/orclcdb"
)
cursor = conn.cursor()

## Cohere API Configuration

In [15]:
cohere_api_key ='uwayrcH0UOQRV7t6SuJUk2oCKRiXhgqzW9D4W84i'

co = cohere.Client(cohere_api_key)

## Reddit scrapping configuration

In [ ]:
subreddits = ["Netherlands"]  #just an example
posts_per_subreddit = 15 
comments_per_post = 5 

## Sentiment Analysis Using VADER with NLTK in Python

In [17]:
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jmsb00nd/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Utils Function For Sentiment Analysis and Content Generation for Subreddits

In [ ]:
#analyze the sentiment of a given text using VADER sentiment analysis
def analyze_sentiment(text):
    sentiment_score = sia.polarity_scores(text)
    compound = sentiment_score["compound"]
    
    if compound >= 0.05:
        return "positive"
    elif compound <= -0.05:
        return "negative"
    else:
        return "neutral"
    

# Function to generate a response using the Cohere API    
def cohere_generate(prompt):
    try:
        response = co.generate(
            model='command-xlarge-nightly',  
            prompt=prompt,
            max_tokens=200
        )
        return response.generations[0].text.strip()
    except Exception as e:
        return f"Error generating response: {str(e)}"
    
# Function to generate a formatted prompt for sentiment analysis based on subreddit posts and comments
def generate_prompt(subreddit_name, posts_with_comments):
    prompt = f"The following is a sentiment analysis for the subreddit '{subreddit_name}' based on selected posts and their comments:\n\n"
    for i, post in enumerate(posts_with_comments, 1):
        prompt += f"Post {i}:\nTitle: {post['post_title']}\nContent: {post['post_content']}\nComments:\n"
        for comment in post["comments"]:
            prompt += f"- {comment}\n"
        prompt += "\n"
    
    prompt += "Please provide a sentiment analysis summarizing the overall tone and themes discussed in this subreddit.\n"
    return prompt

# Function to generate sentiment analysis for a subreddit based on posts and comments from the database
def generate_sentiment_analysis(subreddit_name, subreddit_id, cursor):
    cursor.execute("""
        SELECT post_id, title, content
        FROM Post
        WHERE subreddit_id = :subreddit_id
        FETCH FIRST 10 ROWS ONLY
    """, {"subreddit_id": subreddit_id})
    
    posts = cursor.fetchall()
    if not posts:
        print(f"No posts found for subreddit {subreddit_name}")
        return
    
    selected_posts = random.sample(posts, min(5, len(posts)))
    posts_with_comments = []
    
    for post_id, title, content in selected_posts:
        cursor.execute("""
            SELECT content
            FROM Comments
            WHERE post_id = :post_id
            FETCH FIRST 10 ROWS ONLY
        """, {"post_id": post_id})
        
        comments = [row[0] for row in cursor.fetchall()]
        posts_with_comments.append({
            "post_title": title,
            "post_content": content,
            "comments": comments
        })
    
    prompt = generate_prompt(subreddit_name, posts_with_comments)
    
    sentiment_analysis = cohere_generate(prompt)
    print("Generated Sentiment Analysis:", sentiment_analysis)
    
    cursor.execute("""
        INSERT INTO SubredditSentiments (subreddit_id, sentiment_analysis)
        VALUES (:subreddit_id, :sentiment_analysis)
    """, {"subreddit_id": subreddit_id, "sentiment_analysis": sentiment_analysis})
    
    print(f"Sentiment analysis for subreddit {subreddit_name} saved successfully!")



## Scraping and Analyzing Sentiment from Reddit Posts and Comments

In [22]:
try:
    for subreddit_name in subreddits:
        print(f"Processing subreddit: {subreddit_name}")
        subreddit = reddit.subreddit(subreddit_name)

        subreddit_data = {
            "subreddit_id": subreddit.id,
            "name": subreddit.display_name,
            "description": subreddit.public_description,
            "subscriber_count": subreddit.subscribers,
            "active_users": subreddit.accounts_active,
            "created_at": subreddit.created_utc
        }

        cursor.execute("""
            INSERT INTO Subreddit (subreddit_id, name, description, subscriber_count, active_users, created_at)
            VALUES (:subreddit_id, :name, :description, :subscriber_count, :active_users, TO_DATE('1970-01-01', 'YYYY-MM-DD') + (:created_at / 86400))
        """, subreddit_data)

        for post in subreddit.hot(limit=posts_per_subreddit):
            # Analyze sentiment
            label = analyze_sentiment(post.selftext)
            post_data = {
                "post_id": post.id,
                "title": post.title,
                "content": post.selftext,
                "author": post.author.name if post.author else None,
                "score": post.score,
                "flair": post.link_flair_text,
                "num_comments": post.num_comments,
                "media_url": post.url,
                "subreddit_id": subreddit.id,
                "created_at": post.created_utc,
                "label": label
            }

            cursor.execute("""
                INSERT INTO Post (post_id, title, content, author, score, flair, num_comments, media_url, subreddit_id, created_at, label)
                VALUES (:post_id, :title, :content, :author, :score, :flair, :num_comments, :media_url, :subreddit_id, TO_DATE('1970-01-01', 'YYYY-MM-DD') + (:created_at / 86400), :label)
            """, post_data)

            post.comments.replace_more(limit=0)
            for comment in post.comments[:comments_per_post]:
                # Analyze sentiment for comment content
                label = analyze_sentiment(comment.body)
                comment_data = {
                    "comment_id": comment.id,
                    "content": comment.body,
                    "author": comment.author.name if comment.author else None,
                    "score": comment.score,
                    "post_id": post.id,
                    "parent_comment_id": comment.parent_id[3:] if comment.parent_id.startswith("t1_") else None,
                    "created_at": comment.created_utc,
                    "label": label
                }

                cursor.execute("""
                    INSERT INTO Comments (comment_id, content, author, score, post_id, parent_comment_id, created_at, label)
                    VALUES (:comment_id, :content, :author, :score, :post_id, :parent_comment_id, TO_DATE('1970-01-01', 'YYYY-MM-DD') + (:created_at / 86400), :label)
                """, comment_data)
                
        generate_sentiment_analysis(subreddit_data["name"], subreddit_data["subreddit_id"],cursor)
            
                

    conn.commit()
    print("Data inserted successfully!")

except oracledb.DatabaseError as e:
    print("Error occurred:", e)
    conn.rollback()
finally:
    cursor.close()
    conn.close()


Processing subreddit: Netherlands


Generated Sentiment Analysis: The subreddit 'Netherlands' covers a range of topics related to life in the Netherlands, including consumer warnings, legal issues, immigration, housing, and current events. The overall sentiment is mixed, with some posts expressing frustration and anger, while others provide helpful advice and information.

**Consumer Warnings:**
One post warns Dutch residents about Skare Meat Packers, a Danish meat company that was shut down for selling undocumented, old meat. The post urges people to avoid buying meat from this company, which has now moved its operations to the Netherlands. This post elicits a range of responses, with some users expressing shock and calling for more investigations, while others make dark humor or point out the prevalence of old meat in certain Dutch foods.

**Legal and Employment Issues:**
Several posts deal with legal and employment matters. One user seeks advice after spending €3000 in cash at an electronics store and not receiving th

## (Appendix) Generating Sentiment Analysis for All Subreddits in the Database


In [52]:
def generate_sentiment_analysis_for_all_subreddits(cursor, co):
    try:
        cursor.execute("SELECT subreddit_id, name FROM Subreddit")
        subreddits = cursor.fetchall()
        
        if not subreddits:
            print("No subreddits found in the database.")
            return
        
        for subreddit_id, subreddit_name in subreddits:
            print(f"Processing subreddit: {subreddit_name} (ID: {subreddit_id})")
            
            try:
                generate_sentiment_analysis(subreddit_name, subreddit_id, cursor, co)
            except Exception as e:
                print(f"Error processing subreddit {subreddit_name}: {e}")
        
        print("Sentiment analysis generated for all subreddits.")
    
    except Exception as e:
        print(f"Error fetching subreddits: {e}")


In [53]:
generate_sentiment_analysis_for_all_subreddits(cursor, co)

Processing subreddit: algeria (ID: 2qihd)
Generated Sentiment Analysis: The sentiment analysis of the 'algeria' subreddit reveals a mix of positive, negative, and neutral sentiments, covering various topics related to Algerian culture, society, and personal experiences.

**Positive:**
- Post 1, which features scenic pictures of Algeria, receives praise and appreciation for the visual content. Commenters express enjoyment and find it refreshing to see content beyond the typical English-language Algerian Facebook posts.

**Neutral:**
- Post 4 seeks logistical advice for transportation from Algiers Airport to the city center at night, and commenters provide helpful suggestions and resources, including taxi apps, train schedules, and bus services.

**Negative:**
- Post 2 discusses the topic of insecurity among Algerians, with the author attributing it to societal judgment and strict upbringing. Commenters share personal experiences of criticism, bullying, and abusive environments, suggesti